# <b>S3 Test Notebook</b>

In [ ]:
import os

In [ ]:
# Here you have to set values of environment variables

os.environ['BLAZINGSQL_E2E_AWS_S3_ACCESS_KEY_ID']=''
os.environ['BLAZINGSQL_E2E_AWS_S3_SECRET_KEY']=''

os.environ['BLAZINGSQL_E2E_AWS_S3_VIRGINIA_PUBLIC']=''
os.environ['BLAZINGSQL_E2E_AWS_S3_VIRGINIA_AES']=''
os.environ['BLAZINGSQL_E2E_AWS_S3_VIRGINIA_KMS']=''

os.environ['BLAZINGSQL_E2E_AWS_S3_OREGON_PUBLIC']=''
os.environ['BLAZINGSQL_E2E_AWS_S3_OREGON_AES']=''
os.environ['BLAZINGSQL_E2E_AWS_S3_OREGON_KMS']=''

os.environ['BLAZINGSQL_E2E_AWS_S3_KMS_ID_VIRGINIA']=''
os.environ['BLAZINGSQL_E2E_AWS_S3_KMS_ID_OREGON']=''

In [ ]:
# Getting values of environment variables

access_key_id = os.getenv("BLAZINGSQL_E2E_AWS_S3_ACCESS_KEY_ID")
secret_key = os.getenv("BLAZINGSQL_E2E_AWS_S3_SECRET_KEY")

blazingsql_bucket = os.getenv("BLAZINGSQL_E2E_AWS_S3_VIRGINIA_PUBLIC")
blazingsql_bucket_aes = os.getenv("BLAZINGSQL_E2E_AWS_S3_VIRGINIA_AES")
blazingsql_bucket_aws_kms = os.getenv("BLAZINGSQL_E2E_AWS_S3_VIRGINIA_KMS")

blazingsql_bucket_us_west2 = os.getenv("BLAZINGSQL_E2E_AWS_S3_OREGON_PUBLIC")
blazingsql_bucket_us_west2_aes = os.getenv("BLAZINGSQL_E2E_AWS_S3_OREGON_AES")
blazingsql_bucket_us_west2_aws_kms = os.getenv("BLAZINGSQL_E2E_AWS_S3_OREGON_KMS")

kms_id_virginia = os.getenv("BLAZINGSQL_E2E_AWS_S3_KMS_ID_VIRGINIA")
kms_id_oregon = os.getenv("BLAZINGSQL_E2E_AWS_S3_KMS_ID_OREGON")

## <b>Init Context</b>

In [ ]:
import dask

In [ ]:
from dask.distributed import Client

In [ ]:
ip_port_dask_scheduller = 'SCHEDULER_IP:8786'
network_interface = 'ens5' 

In [ ]:
client = Client(ip_port_dask_scheduller)

In [ ]:
client

In [ ]:
from blazingsql import BlazingContext

In [ ]:
bc = BlazingContext(dask_client = client, network_interface='ens5')

## <b>Register and create tables in Virginia region</b>

<b>Buckets</b>

* blazingsql-bucket
* blazingsql-bucket-aes-256
* blazingsql-bucket-aws-kms

<b>Tablas</b>
* nation
* region
* customer
* lineitem

In [ ]:
from blazingsql import S3EncryptionType

In [ ]:
authority = "tpch_s3_virginia"

In [ ]:
bc.s3(authority, bucket_name=blazingsql_bucket, encryption_type=S3EncryptionType.NONE,
      access_key_id=access_key_id, secret_key=secret_key)

In [ ]:
dir_data_lc = "s3://" + authority + "/" + "DataSet100Mb2part/" + "tpch/"

In [ ]:
ext = "parquet"

In [ ]:
table_files_nation = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "nation", ext)

In [ ]:
table_files_region = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "region", ext)

In [ ]:
print("nation files: " + table_files_nation)

In [ ]:
bc.create_table("nation", table_files_nation)

In [ ]:
bc.create_table("region", table_files_region)

In [ ]:
query = """select n1.n_nationkey as n1key, n2.n_nationkey as n2key, n1.n_nationkey + n2.n_nationkey 
from nation as n1 full outer join nation as n2 on n1.n_nationkey = n2.n_nationkey + 6 
where n1.n_nationkey < 10 and n1.n_nationkey > 5"""

In [ ]:
result = bc.sql(query)

In [ ]:
print(result.compute())

In [ ]:
authority = "tpch_s3_virginia_aes"

In [ ]:
bc.s3(authority, bucket_name=blazingsql_bucket_aes,
      access_key_id=access_key_id,
      secret_key=secret_key,
      encryption_type=S3EncryptionType.AES_256)

In [ ]:
dir_data_lc = "s3://" + authority + "/" + "DataSet100Mb2part/" + "tpch/"

In [ ]:
table_files_customer = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "customer", ext)

In [ ]:
print("customer files: " + table_files_customer)

In [ ]:
bc.create_table("customer", table_files_customer)
print("customer table created!")

In [ ]:
query = """select * from customer limit 10"""

In [ ]:
result = bc.sql(query)

In [ ]:
print(result.compute())

In [ ]:
authority = "tpch_s3_virginia_kms"

In [ ]:
bc.s3(authority, bucket_name=blazingsql_bucket_aws_kms,
      access_key_id=access_key_id,
      secret_key=secret_key,
      encryption_type=S3EncryptionType.AWS_KMS,
      kms_key_amazon_resource_name=kms_id_virginia)

In [ ]:
dir_data_lc = "s3://" + authority + "/" + "DataSet100Mb2part/" + "tpch/"

In [ ]:
table_files_lineitem = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "lineitem", ext)

In [ ]:
bc.create_table("lineitem", table_files_lineitem)
print("lineitem table created!")

In [ ]:
query = """select * from lineitem limit 10"""

In [ ]:
result = bc.sql(query)

In [ ]:
print(result.compute())

## <b>Register and create tables in Oregon region</b>

<b>Buckets</b>

* blazingsql-bucket-us-west-2
* blazingsql-bucket-us-west-2-aes-256
* blazingsql-bucket-us-west-2-aws-kms

<b>Tablas</b>

* part
* partsupp
* supplier

In [ ]:
authority = "tpch_s3_oregon"

In [ ]:
bc.s3(authority, bucket_name=blazingsql_bucket_us_west2, 
      encryption_type=S3EncryptionType.NONE,
      access_key_id=access_key_id,
      secret_key=secret_key)

In [ ]:
dir_data_lc = "s3://" + authority + "/" + "DataSet100Mb2part/" + "tpch/"

In [ ]:
table_files_part = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "part", ext)

In [ ]:
print("part files: " + table_files_part)

In [ ]:
bc.create_table("part", table_files_part)
print("part table created!")

In [ ]:
query = "select * from part limit 20"

In [ ]:
result = bc.sql(query)

In [ ]:
print(result.compute())

In [ ]:
authority = "tpch_s3_oregon_aes"

In [ ]:
bc.s3(authority, bucket_name=blazingsql_bucket_us_west2_aes,
      access_key_id=access_key_id,
      secret_key=secret_key,
      encryption_type=S3EncryptionType.AES_256)

In [ ]:
dir_data_lc = "s3://" + authority + "/" + "DataSet100Mb2part/" + "tpch/"

In [ ]:
table_files_partsupp = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "partsupp", ext)

In [ ]:
print("partsupp files: " + table_files_partsupp)

In [ ]:
bc.create_table("partsupp", table_files_partsupp)
print("partsupp table created!")

In [ ]:
query = """select * from partsupp limit 10"""

In [ ]:
result = bc.sql(query)

In [ ]:
print(result.compute())

In [ ]:
authority = "tpch_s3_oregon_kms"

In [ ]:
bc.s3(authority, bucket_name=blazingsql_bucket_us_west2_aws_kms,
      access_key_id=access_key_id,
      secret_key=secret_key,
      encryption_type=S3EncryptionType.AWS_KMS,
      kms_key_amazon_resource_name=kms_id_oregon)

In [ ]:
dir_data_lc = "s3://" + authority + "/" + "DataSet100Mb2part/" + "tpch/"

In [ ]:
table_files_orders = ("%s/%s_[0-9]*.%s") % (dir_data_lc, "orders", ext)

In [ ]:
print("orders files: " + table_files_orders)

In [ ]:
bc.create_table("orders", table_files_orders)
print("orders table created!")

In [ ]:
query = """select * from orders limit 20"""

In [ ]:
result = bc.sql(query)

In [ ]:
print(result.compute())

# <b>Queries with created tables with data from different regions</b>

In [ ]:
query = """
                select
                    l.l_orderkey,
                    sum(l.l_extendedprice*(1-l.l_discount)) as revenue,
                    o.o_orderdate,
                    o.o_shippriority
                from
                    customer c
                    inner join orders o 
                    on c.c_custkey = o.o_custkey
                    inner join lineitem l
                    on l.l_orderkey = o.o_orderkey
                where
                    c.c_mktsegment = 'BUILDING'
                    and o.o_orderdate < date '1995-03-15'
                    and l.l_shipdate > date '1995-03-15'
                group by
                    l.l_orderkey,
                    o.o_orderdate,
                    o.o_shippriority
                order by
                    revenue desc,
                    o.o_orderdate
                limit 10
            """

In [ ]:
result = bc.sql(query)

In [ ]:
print(result.compute())

In [ ]:
query = """
                select
                    c.c_custkey,
                    c.c_name,
                    sum(l.l_extendedprice * (1 - l.l_discount)) as revenue,
                    c.c_acctbal,
                    n.n_name,
                    c.c_address,
                    c.c_phone,
                    c.c_comment
                from
                    customer c
                    inner join orders o
                    on c.c_custkey = o.o_custkey
                    inner join lineitem l
                    on l.l_orderkey = o.o_orderkey
                    inner join nation n
                    on c.c_nationkey = n.n_nationkey
                where
                    o.o_orderdate >= date '1993-10-01'
                    and o.o_orderdate < date '1993-10-01' + interval '3' month
                    and l.l_returnflag = 'R'
                group by
                    c.c_custkey,
                    c.c_name,
                    c.c_acctbal,
                    c.c_phone,
                    n.n_name,
                    c.c_address,
                    c.c_comment
                order by
                    revenue desc
                limit 20
            """

In [ ]:
result = bc.sql(query)

In [ ]:
print(result.compute())